In [1]:
import keras
from keras import models
from keras import layers
keras.__version__

Using TensorFlow backend.


'2.3.1'

In [2]:
# import and "clean" data

In [4]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical ## we need that as our labels cannot be represented by a single output node

base_dir = 'Course notebooks\paintings'

train_dir = os.path.join(base_dir, 'train')
test_dir = os.path.join(base_dir, 'test')
validation_dir = os.path.join(base_dir, 'validation')

total_no_of_training_images = 210  ## Keras seems to lack the ability to count, so we set this manually:
total_no_of_test_images = 30
total_no_of_validation_images = 60

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 5
input_width = 150
input_height = 150
input_area = input_width * input_height
input_shape = (input_width, input_height, 3) # 3 for color

In [9]:
def extract_features(directory, sample_count):
    data = [] #np.zeros(shape=(sample_count))
    labels = [] #np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(input_width, input_height),
        batch_size=batch_size)#,
        #class_mode='sparse')   ## NB: the documentation's explanation of class_mode is unclear,
                               ## and the setting 'sparse' has been found by experimentation

    i = 0
    for inputs_batch, labels_batch in generator:
        data[i * batch_size : (i + 1) * batch_size] = inputs_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    labels = to_categorical(labels)
    return data, labels


In [10]:
train_data, train_labels = extract_features(train_dir, total_no_of_training_images)
validation_data, validation_labels = extract_features(validation_dir, total_no_of_validation_images)
test_data, test_labels = extract_features(test_dir, total_no_of_test_images)

Found 210 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 30 images belonging to 3 classes.


In [21]:
train_data = np.reshape(train_data, total_no_of_training_images * 67500)
validation_data = np.reshape(validation_data, total_no_of_validation_images * 67500) #### WTF
test_data = np.reshape(test_data, total_no_of_test_images * 67500)

In [22]:
train_data.shape()

TypeError: 'tuple' object is not callable

In [23]:
# Define model

In [31]:
model = models.Sequential()
model.add(layers.Dense(512, activation='relu', input_shape= (14175000,1) ))
model.add(layers.Dense(3, activation='sigmoid'))

In [32]:
# train model

In [33]:
epochs = 1

model.compile(loss="categorical_crossentropy", optimizer="RMSprop", metrics=["acc"])

model.fit(train_data, train_labels, batch_size=5, epochs=epochs, validation_split=0.1)

KeyboardInterrupt: 

In [6]:
# validation